In [ ]:
%%writefile assign3.cu

#include <stdio.h>
#define N 5

// global static memory array
__device__ int Cglobal[N];


//__global__ void addArray(const int *A, const int *B)
{
  int idx = threadIdx.x;

  if (idx < N)
  {
      Cglobal[idx] = A[idx] + B[idx];
  }
}


void initializeGlobal_C(int *hostArray){
   cudaMemcpyToSymbol(Cglobal, hostArray, N*sizeof(int), 0, cudaMemcpyHostToDevice);
}


void retrieveGlobalArray(int *hostArray){
   cudaMemcpyFromSymbol(hostArray, Cglobal, N*sizeof(int), 0, cudaMemcpyDeviceToHost);
}



int main()
{

  // Initialize the host array A and B
  int A[N]={1,2,3,4,5};
  int B[N]={5,4,3,2,1};
  int C[N];

  int *A_device, *B_device;

  cudaMalloc(&A_device, N*sizeof(int));
  cudaMalloc(&B_device, N*sizeof(int));

  cudaMemcpy(A_device, A, N*sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(B_device, B, N*sizeof(int), cudaMemcpyHostToDevice);

  addArray<<<1,N>>>(A_device, B_device);

  cudaDeviceSynchronize();

  retrieveGlobalArray(C);

  printf("Modified globalArray: \n");
  for (int i=0; i<N; i++)
  {
    printf("%d ",C[i]);
  }

  printf("\n");

  return 0;
}

Writing assign3.cu


In [ ]:
!nvcc -o exe assign3.cu
!./exe

Modified globalArray: 
6 6 6 6 6 
